In [2]:
import pandas as pd

In [45]:
archive_location = 'archive'

filenames = {
    "customers" : "olist_customers_dataset.csv", #Information about customers and their geographic data.
    "geolocation" : "olist_geolocation_dataset.csv",
    "order_items" : "olist_order_items_dataset.csv", #Details about each item in an order, including price, freight value, and the connection between orders and products.
    "order_payments" : "olist_order_payments_dataset.csv", #Payment information associated with each order.
    "order_reviews" : "olist_order_reviews_dataset.csv", #Customer reviews and ratings for orders.
    "orders" : "olist_orders_dataset.csv", #Contains order information such as order status, purchase date, and delivery date.
    "products" : "olist_products_dataset.csv", #Contains product-related information.
    "sellers" : "olist_sellers_dataset.csv", #Details about sellers and their geographic information.
    "product_category_name_translation" : "product_category_name_translation.csv",
}
customers_df = pd.read_csv(f'{archive_location}/{filenames["customers"]}')
geolocation_df = pd.read_csv(f'{archive_location}/{filenames["geolocation"]}')
order_items = pd.read_csv(f'{archive_location}/{filenames["order_items"]}')
order_payments_df = pd.read_csv(f'{archive_location}/{filenames["order_payments"]}')
order_reviews = pd.read_csv(f'{archive_location}/{filenames["order_reviews"]}')
orders_df = pd.read_csv(f'{archive_location}/{filenames["orders"]}')
products = pd.read_csv(f'{archive_location}/{filenames["products"]}')
sellers = pd.read_csv(f'{archive_location}/{filenames["sellers"]}')
product_category_name_translation = pd.read_csv(f'{archive_location}/{filenames["product_category_name_translation"]}')

# print(f"Count of rows before dedublication: {str(len(geolocation_df.index))}")
geolocation_df.drop_duplicates(inplace=True)
geolocation_df.reset_index(inplace=True , drop=True)
# print(f"Count of rows after dedublication: {len(geolocation_df.index)}")

# print(orders_df[orders_df["order_approved_at"].isna()].head())
orders_df["order_delivered_carrier_date"] = orders_df["order_delivered_carrier_date"].fillna(pd.NaT)
orders_df["order_approved_at"] = orders_df["order_approved_at"].fillna(pd.NaT)
orders_df["order_delivered_customer_date"] = orders_df["order_delivered_customer_date"].fillna(pd.NaT)
# print(orders_df[orders_df["order_approved_at"].isna()].head())


# print(products[products["product_category_name"].isna()].head())
products["product_category_name"] = products["product_category_name"].fillna('')
products["product_name_lenght"] = products["product_category_name"].fillna('')
products["product_description_lenght"] = products["product_description_lenght"].fillna(0)
products["product_photos_qty"] = products["product_photos_qty"].fillna(0)
products["product_weight_g"] = products["product_weight_g"].fillna('')
products["product_length_cm"] = products["product_length_cm"].fillna('')
products["product_photos_qty"] = products["product_photos_qty"].fillna('')
products["product_width_cm"] = products["product_width_cm"].fillna('')
# print('\n\n----------------')
# print(products[products["product_category_name"] == ''].head())


# print(order_reviews[order_reviews["review_comment_message"].isna()].head())
order_reviews["review_comment_title"] = order_reviews["review_comment_title"].fillna('')
order_reviews["review_comment_message"] = order_reviews["review_comment_message"].fillna('')
# print(order_reviews[order_reviews["review_comment_message"] == ''].head())


# print("Calculating total_price")
order_items["total_price"] = order_items["freight_value"] +  order_items["price"]
# print(order_items[["freight_value","price","total_price"]].head())


# print("\n\nCalculating delivery_time")
orders_df["delivery_time"] = pd.to_datetime(orders_df["order_delivered_customer_date"]) - pd.to_datetime(orders_df["order_purchase_timestamp"])
# print(orders_df[["order_delivered_customer_date","order_purchase_timestamp","delivery_time"]].head())

# print("\n\nAdding delivery_time to order_items")
order_items = pd.merge(order_items, orders_df[['order_id', 'delivery_time']], on='order_id', how='left')
# print(order_items['delivery_time'].head())

#tbc on negative values
# print("\n\nCalculating profit_margin")
order_items["profit_margin"] = order_items["price"] - order_items["freight_value"]
# print(order_items[["price","freight_value","profit_margin"]].sort_values(by=["profit_margin"],ascending=True).head())

# print("\n\nCalculating profit_margin")
payments_count = order_payments_df.groupby('order_id').size().reset_index(name='payments_count')
orders_df = orders_df.merge(payments_count, on='order_id', how='left')
orders_df['payments_count'] = orders_df['payments_count'].fillna(0)
orders_df['payments_count'] = orders_df['payments_count'].astype(int)

# print(orders_df['payments_count'].head())



In [46]:
orders_with_price_df = pd.merge(orders_df, order_items[['order_id', 'total_price']], on='order_id', how='left')

orders_with_price_df['running_total'] = orders_with_price_df.groupby('customer_id')['total_price'].cumsum()

#to show the running total of product price for each customer
testorders_with_price_df = orders_with_price_df[orders_with_price_df["customer_id"] == '1617b1357756262bfa56ab541c47bc16']
print(testorders_with_price_df[["customer_id","total_price","running_total"]].head())


                            customer_id  total_price  running_total
15178  1617b1357756262bfa56ab541c47bc16      1708.01        1708.01
15179  1617b1357756262bfa56ab541c47bc16      1708.01        3416.02
15180  1617b1357756262bfa56ab541c47bc16      1708.01        5124.03
15181  1617b1357756262bfa56ab541c47bc16      1708.01        6832.04
15182  1617b1357756262bfa56ab541c47bc16      1708.01        8540.05


In [47]:

orders_df['delivery_time'] = pd.to_numeric(orders_df['delivery_time'], errors='coerce')

orders_with_productId_df = pd.merge(orders_df, order_items[['order_id', 'product_id']], on='order_id', how='left')
# print(orders_with_productId_df.head())
orders_with_product_category_df = pd.merge(orders_with_productId_df, products[['product_id', 'product_category_name']], on='product_id', how='left')
# print(orders_with_product_category_df.head())

orders_with_product_category_df['rolling_average_delivery_time'] = orders_with_product_category_df.groupby('product_category_name')['delivery_time'].rolling(window=2).mean().reset_index(level=0, drop=True)
orders_with_product_category_df['delivery_time'] = pd.to_timedelta(orders_with_product_category_df['delivery_time'])
orders_with_product_category_df['rolling_average_delivery_time'] = pd.to_timedelta(orders_with_product_category_df['rolling_average_delivery_time'])

#to show the rolling of the avg delivery time
testorders_with_product_category_df = orders_with_product_category_df[orders_with_product_category_df["product_category_name"] == "utilidades_domesticas"]
print(testorders_with_product_category_df[['product_category_name',"delivery_time","rolling_average_delivery_time"]].head())

    product_category_name    delivery_time rolling_average_delivery_time
0   utilidades_domesticas  8 days 10:28:40                           NaT
43  utilidades_domesticas 33 days 21:31:35       21 days 04:00:07.500000
83  utilidades_domesticas 19 days 16:02:09              26 days 18:46:52
84  utilidades_domesticas 19 days 16:02:09              19 days 16:02:09
85  utilidades_domesticas 19 days 16:02:09              19 days 16:02:09
